# Age and Gender prediction using Deep Learning 

***Importing the necessary libraries***


In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf


In [2]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,Flatten,Dense,Input,Dropout
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.preprocessing import image as img_preProcess

***Applying Neural network***


In [3]:
dropout = Dropout(0.1)
inp = Input((248,248,3))

model=EfficientNetB1(include_top=False,weights='imagenet',input_shape=(248,248,3))(inp)
model=Conv2D(2000, kernel_size=7,activation='swish')(model)
model=Flatten()(model)
model=Dense(2000,activation='swish')(model)
oupt1=Dense(2,activation='softmax',name='gen')(model)
oupt2=Dense(1,activation='relu',name='age')(model)
model = Model(inp, [oupt1,oupt2])



2022-06-29 14:32:22.086728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 14:32:22.225128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 14:32:22.226299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 14:32:22.228225: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

27033600/27018416 [==============================] - 1s 0us/step


***Compiling the model***


In [5]:
model.compile(loss={'gen':'sparse_categorical_crossentropy','age':'mean_absolute_error'}, 
                    
              optimizer='adam',
              metrics={'gen':'accuracy','age':'mae'})


***Summarising the layers and images shapes***


In [6]:
model.summary()
tf.config.run_functions_eagerly(True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 248, 248, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb1 (Functional)     (None, 8, 8, 1280)   6575239     input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 2, 2, 2000)   125442000   efficientnetb1[0][0]             
__________________________________________________________________________________________________
flatten (Flatten)               (None, 8000)         0           conv2d[0][0]                     
______________________________________________________________________________________________

***Running the epochs***


In [7]:
for i in range(2):
    print('epoch',i)
    for i in range(0,23000,1000):
        path = "../input/utkface-new/UTKFace/"
        loc = os.listdir(path)
        loc=loc[i:i+1000]
        data = []
        age = []
        gender = []
        #[0=MAle , 1=Female]
        for i in loc:
            img = cv2.imread(path+i)
            img = cv2.resize(img,(248,248))
            data.append(img)
            atributes= i.split('_')
            age.append(float(atributes[0]))
            gender.append(int(atributes[1]))
        data=np.array(data)
        age=np.array(age)
        gender=np.array(gender)
        
        model.fit(data,[gender,age],epochs=1,batch_size=32)
        

epoch 0


/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "
2022-06-29 14:32:35.872332: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-29 14:32:36.943032: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


32/32 [==============================] - 26s 805ms/step - loss: 6.9484 - gen_loss: 0.3060 - age_loss: 6.6424 - gen_accuracy: 0.8800 - age_mae: 6.6424
epoch 1
32/32 [==============================] - 25s 787ms/step - loss: 5.7244 - gen_loss: 0.2434 - age_loss: 5.4810 - gen_accuracy: 0.8960 - age_mae: 5.4810


***Saving the model***

In [8]:
model.save('md1.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
